In [12]:
from dataset import extract_stage3_ground_truth
from utils import parse_panels, bbox_to_xyxy, parse_rules
import xml.etree.ElementTree as ET
import os
from pprint import pprint
import torch

from torch.utils.data import Dataset

In [2]:
panels, rules, targets = extract_stage3_ground_truth('dataset', 'train', ['center_single', 'distribute_four'], False)

In [9]:
panels.merge(rules, on='file').merge(targets, on='file').drop(columns=['component'])

,file,panel,slot0_color,slot0_size,slot0_type,slot1_color,slot1_size,slot1_type,slot2_color,slot2_size,...,slot3_type,slot4_color,slot4_size,slot4_type,component0_number,component0_position,component0_type,component0_size,component0_color,target
0,dataset/center_single/RAVEN_53_train,6,6,1,4,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,Constant,Constant,Distribute_Three,Distribute_Three,Distribute_Three,4
1,dataset/center_single/RAVEN_53_train,7,3,0,3,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,Constant,Constant,Distribute_Three,Distribute_Three,Distribute_Three,4
2,dataset/center_single/RAVEN_53_train,8,7,5,3,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,Constant,Constant,Distribute_Three,Distribute_Three,Distribute_Three,4
3,dataset/center_single/RAVEN_53_train,9,2,2,3,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,Constant,Constant,Distribute_Three,Distribute_Three,Distribute_Three,4
4,dataset/center_single/RAVEN_53_train,10,2,2,5,-1,-1,-1,-1,-1,...,-1,-1,-1,-1,Constant,Constant,Distribute_Three,Distribute_Three,Distribute_Three,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11995,dataset/distribute_four/RAVEN_14_train,11,-1,-1,-1,-1,-1,-1,9,3,...,3,-1,-1,-1,Progression,Progression,Distribute_Three,Arithmetic,Distribute_Three,6
11996,dataset/distribute_four/RAVEN_14_train,12,-1,-1,-1,9,3,3,9,3,...,3,-1,-1,-1,Progression,Progression,Distribute_Three,Arithmetic,Distribute_Three,6
11997,dataset/distribute_four/RAVEN_14_train,13,-1,-1,-1,9,2,3,9,2,...,3,-1,-1,-1,Progression,Progression,Distribute_Three,Arithmetic,Distribute_Three,6
11998,dataset/distribute_four/RAVEN_14_train,14,-1,-1,-1,-1,-1,-1,0,3,...,3,-1,-1,-1,Progression,Progression,Distribute_Three,Arithmetic,Distribute_Three,6


In [10]:
rules

,file,component0_number,component0_position,component0_type,component0_size,component0_color
0,dataset/center_single/RAVEN_53_train,Constant,Constant,Distribute_Three,Distribute_Three,Distribute_Three
1,dataset/center_single/RAVEN_150_train,Constant,Constant,Progression,Progression,Distribute_Three
2,dataset/center_single/RAVEN_961_train,Constant,Constant,Distribute_Three,Arithmetic,Constant
3,dataset/center_single/RAVEN_685_train,Constant,Constant,Progression,Constant,Progression
4,dataset/center_single/RAVEN_220_train,Constant,Constant,Progression,Distribute_Three,Progression
...,...,...,...,...,...,...
1195,dataset/distribute_four/RAVEN_501_train,Distribute_Three,Distribute_Three,Constant,Distribute_Three,Distribute_Three
1196,dataset/distribute_four/RAVEN_784_train,Arithmetic,Arithmetic,Progression,Constant,Progression
1197,dataset/distribute_four/RAVEN_860_train,Arithmetic,Arithmetic,Constant,Progression,Constant
1198,dataset/distribute_four/RAVEN_321_train,Distribute_Three,Distribute_Three,Progression,Progression,Arithmetic


In [85]:
class AVRDatasetSimple(Dataset):
    def __init__(self, dataset_dir, split, configs):
        super().__init__()
        panels, rules, targets = extract_stage3_ground_truth(dataset_dir, split, configs, False)
        self.panels = panels
        self.rule2id = {'Constant': 0, 'Distribute_Three': 1, 'Progression': 2, 'Arithmetic': 3}
        self.rules = rules.replace(self.rule2id).set_index('file')
        self.targets = targets.set_index('file')
        self.files = self.targets.index.to_list()
    
    def __len__(self):
        return len(self.files)

    def __getitem__(self, index):
        filename = self.files[index]
        panels = self.panels[self.panels['file'] == filename].drop(columns=['file', 'panel', 'component'])
        panels_tensor = torch.tensor(panels.values)
        rules = self.rules.loc[filename]
        rules_tensor = torch.tensor(rules.values)
        target = self.targets.loc[filename]
        target_tensor = torch.tensor(target.values[0])

        return {'panels': panels_tensor,
                'rules': rules_tensor,
                'target': target_tensor}

In [86]:
dataset = AVRDatasetSimple('dataset', 'train', ['distribute_four'])

In [87]:
dataset[0]

{'panels': tensor([[-1, -1, -1, -1, -1, -1, -1, -1, -1,  0,  2,  5],
         [-1, -1, -1, -1, -1, -1,  4,  0,  3,  4,  0,  3],
         [ 0,  3,  2, -1, -1, -1,  0,  3,  2, -1, -1, -1],
         [ 3,  3,  2,  3,  3,  2, -1, -1, -1,  3,  3,  2],
         [ 3,  3,  1, -1, -1, -1,  3,  3,  1, -1, -1, -1],
         [ 3,  3,  1,  3,  3,  1, -1, -1, -1,  3,  3,  1],
         [ 0,  3,  2,  0,  3,  2, -1, -1, -1,  0,  3,  2],
         [ 0,  3,  1, -1, -1, -1,  0,  3,  1, -1, -1, -1],
         [ 3,  3,  2, -1, -1, -1,  3,  3,  2, -1, -1, -1],
         [ 0,  3,  1,  0,  3,  1, -1, -1, -1,  0,  3,  1]]),
 'rules': tensor([3, 3, 2, 3, 1]),
 'target': tensor(3)}